<a href="https://colab.research.google.com/github/asmaaitnasser/DDDM/blob/main/Prediction_Ble_Region_Fusion_Filtr%C3%A9e.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🌾 Prédiction Fiable de la Production de Blé au Maroc (avec Fusion Filtrée)

Ce notebook utilise les données météo multi-régionales, fusionnées avec la production et la superficie de blé,
en filtrant uniquement les lignes pour lesquelles les données **existent dans toutes les sources** (`_merge == 'both'`).


In [ ]:

import pandas as pd
import numpy as np
import unicodedata
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error


In [ ]:

# Uploader dans Colab :
paths = {
    "autres": "/content/meteom.xlsx",
    "meknes": "/content/meteomeknes.xlsx",
    "kech": "/content/meteokech.xlsx",
    "fes": "/content/meteofes.xlsx",
    "rabat": "/content/meteorabat.xlsx",
    "agadir": "/content/meteo.xlsx",
    "prod": "/content/20230714-production-vegetale-2010-2022.xlsx"
}


In [ ]:

def normalize_ville(v):
    return unicodedata.normalize("NFKD", str(v).strip().lower()).encode("ascii", "ignore").decode("utf-8")

def clean_meteo(df, ville):
    df = df.copy()
    df["annee"] = df["annee"].astype(str).str.extract(r"(\d{4})").astype(int)
    df["ville"] = ville
    for col in df.columns:
        if any(u in col.lower() for u in ["temp", "vent", "précip", "humidité", "visib", "couverture"]):
            df[col] = df[col].astype(str).str.replace(r"[^\d.]", "", regex=True).replace("", "0").astype(float)
    df["ville"] = df["ville"].apply(normalize_ville)
    return df

# Charger tous les fichiers météo
dfs = {key: clean_meteo(pd.read_excel(paths[key]), key) for key in ["meknes", "kech", "fes", "rabat"]}
dfs["agadir"] = clean_meteo(pd.read_excel(paths["agadir"]), "agadir")

df_autres = pd.read_excel(paths["autres"])
df_autres["annee"] = df_autres["annee"].astype(str).str.extract(r"(\d{4})").astype(int)
df_autres["ville"] = df_autres["ville"].astype(str).str.extract(r"à\s+(.*)$")[0].str.lower()
df_autres = clean_meteo(df_autres, df_autres["ville"])

df_all = pd.concat([*dfs.values(), df_autres], ignore_index=True)


In [ ]:

df_ble = pd.read_excel(paths["prod"], sheet_name="Feuil1")
df_ble = df_ble.rename(columns={" Valeur": "valeur"})
df_ble["annee"] = df_ble["Occurrence"].astype(str).str.extract(r"(\d{4})").astype(int)

df_prod = df_ble[
    (df_ble["Filière"].str.lower() == "céréales") &
    (df_ble["Produit"].str.lower().isin(["blé tendre", "blé dur"])) &
    (df_ble["Indicateur"].str.lower().str.contains("production"))
].copy()

df_sup = df_ble[
    (df_ble["Filière"].str.lower() == "céréales") &
    (df_ble["Produit"].str.lower().isin(["blé tendre", "blé dur"])) &
    (df_ble["Indicateur"].str.lower().str.contains("superficie"))
].copy()

# Étendre sur toutes les villes normalisées
villes = df_all["ville"].unique()
df_prod = pd.concat([df_prod.assign(ville=normalize_ville(v)) for v in villes], ignore_index=True)
df_sup = pd.concat([df_sup.assign(ville=normalize_ville(v)) for v in villes], ignore_index=True)

df_prod = df_prod.groupby(["annee", "ville"])["valeur"].sum().reset_index(name="production")
df_sup = df_sup.groupby(["annee", "ville"])["valeur"].sum().reset_index(name="superficie")


In [ ]:

# Filtrer uniquement les lignes avec données météo ET production
df_test = df_all.merge(df_prod, on=["annee", "ville"], how="outer", indicator=True)
df_final = df_test[df_test["_merge"] == "both"].drop(columns=["_merge"])

# Ajouter superficie (seulement pour ceux filtrés)
df_final = df_final.merge(df_sup, on=["annee", "ville"], how="left")
df_final["rendement"] = df_final["production"] / df_final["superficie"]
df_final["superficie"] = df_final["superficie"].fillna(df_final["superficie"].mean())
df_final["rendement"] = df_final["production"] / df_final["superficie"]



In [ ]:

X = df_final.drop(columns=["annee", "ville", "production", "rendement"])
y = df_final["production"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lr = LinearRegression().fit(X_train, y_train)
rf = RandomForestRegressor(n_estimators=100, random_state=42).fit(X_train, y_train)

y_pred_lr = lr.predict(X_test)
y_pred_rf = rf.predict(X_test)


ValueError: could not convert string to float: '1746111073-93'

In [ ]:

def eval_model(name, y_true, y_pred):
    print(f"📌 {name}")
    print(f"R²   : {r2_score(y_true, y_pred):.2f}")
    print(f"RMSE : {np.sqrt(mean_squared_error(y_true, y_pred)):.0f}")
    print(f"MAE  : {mean_absolute_error(y_true, y_pred):.0f}\n")

eval_model("Régression Linéaire", y_test, y_pred_lr)
eval_model("Random Forest", y_test, y_pred_rf)
